In [1]:
import pandas as pd
import numpy as np
import urllib.request, json
from pandas.io.json import json_normalize
from sklearn.ensemble import RandomForestClassifier

In [16]:
matches = pd.read_csv('table with matches patch 7.06.csv', index_col=0, encoding='latin-1')
len(matches) 

3773

In [17]:
matches = matches.dropna()
matches

,id matches,patch,region,league_name,league_id,radiant_name,radiant_id,dire_name,dire_id,radiant_score,...,ban10,pick5,pick6,pick7,pick8,ban11,ban12,pick9,pick10,radiant_win
0,3186238661,20.0,12,WCA Tournament Platform Classic,5418,CAVALRY,70.0,Rock Gaming,59.0,23,...,17,28,106,65,66,63,11,104,70,1
0,3186335263,20.0,12,WCA Tournament Platform Classic,5418,Rock Gaming,59.0,CAVALRY,70.0,30,...,18,54,29,74,95,39,2,11,55,0
0,3188134419,20.0,12,SL i-League Invitational Season 2,5364,TNC Pro Team,29.0,Invictus Gaming,23.0,25,...,46,77,55,88,49,18,98,48,76,1
0,3188202852,20.0,12,SL i-League Invitational Season 2,5364,Invictus Gaming,23.0,TNC Pro Team,29.0,19,...,107,28,87,95,34,65,17,33,25,0
0,3189341121,20.0,3,DreamLeague season 7,5336,Virtus.pro,18.0,Elements Pro Gaming,33.0,24,...,65,20,92,10,46,70,39,93,17,1
0,3189457454,20.0,3,DreamLeague season 7,5336,Elements Pro Gaming,33.0,Virtus.pro,18.0,34,...,46,63,11,6,51,70,74,77,9,1
0,3189558500,20.0,3,DreamLeague season 7,5336,Natus Vincere,17.0,mousesports,57.0,30,...,11,107,28,109,63,106,17,76,25,1
0,3189616858,20.0,3,DreamLeague season 7,5336,Natus Vincere,17.0,mousesports,57.0,29,...,107,25,38,65,60,106,93,13,10,0
0,3190190649,20.0,12,WCA Tournament Platform Classic,5418,Team Braveheart,51.0,TEAM MAX,44.0,9,...,76,8,83,25,87,18,57,54,108,0
0,3190258651,20.0,12,WCA Tournament Platform Classic,5418,TEAM MAX,44.0,Team Braveheart,51.0,38,...,70,37,34,16,48,65,54,83,67,1


In [18]:
pd.value_counts(matches['league_name'])

The International 2017                                                 298
SL i-League Invitational Season 3                                      190
Perfect World Masters                                                  167
ESL One Hamburg 2017                                                   110
Dota 2 Professional League 2016                                         95
PGL DOTA2 OPEN                                                          83
AMD SAPPHIRE Dota PIT                                                   79
DreamLeague season 7                                                    78
EPICENTER: Moscow Season 2                                              59
The Summit 5                                                            55
2017 Mars Dota 2 League                                                 53
ProDotA Cup by Azubu                                                    47
D2CL Season 7                                                           42
Uprise Champions Cup: Spe

In [20]:
# создать DF с двумя командами и их пиком героев
def create_df_team_picks(matches):
    team_picks = pd.DataFrame(columns=['radiant_id','radiant_name', 'dire_id','dire_name', 'kills', 'radiant1', 'radiant2','radiant3','radiant4','radiant5',
                                      'dire1','dire2','dire3','dire4','dire5', 'radiant_win'])
    for index, first_pick in enumerate(matches['first_ban_radiant']):
        if first_pick == 1:
            team_picks['radiant_id'] = matches['radiant_id']
            team_picks['radiant_name'] = matches['radiant_name']
            team_picks['dire_id'] = matches['dire_id']
            team_picks['dire_name'] = matches['dire_name']
            team_picks['kills'] = matches['radiant_score'] + matches['dire_score']
            team_picks['radiant1'] = matches['pick1']
            team_picks['radiant2'] = matches['pick4']
            team_picks['radiant3'] = matches['pick6']
            team_picks['radiant4'] = matches['pick8']
            team_picks['radiant5'] = matches['pick9']
            team_picks['dire1'] = matches['pick2']
            team_picks['dire2'] = matches['pick3']
            team_picks['dire3'] = matches['pick5']
            team_picks['dire4'] = matches['pick7']
            team_picks['dire5'] = matches['pick10']
            team_picks['radiant_win'] = matches['radiant_win']
            #a = pd.DataFrame(matches[['radiant_id', 'dire_id', 'pick1','pick4','pick6','pick8','pick9',
            #                          'pick2','pick3','pick5','pick8','pick10',]].iloc[index])
            
            #print(a)
        else:
            team_picks['radiant_id'] = matches['radiant_id']
            team_picks['radiant_name'] = matches['radiant_name']
            team_picks['dire_id'] = matches['dire_id']
            team_picks['dire_name'] = matches['dire_name']
            team_picks['kills'] = matches['radiant_score'] + matches['dire_score']
            team_picks['radiant1'] = matches['pick2']
            team_picks['radiant2'] = matches['pick3']
            team_picks['radiant3'] = matches['pick5']
            team_picks['radiant4'] = matches['pick8']
            team_picks['radiant5'] = matches['pick10']
            team_picks['dire1'] = matches['pick1']
            team_picks['dire2'] = matches['pick4']
            team_picks['dire3'] = matches['pick6']
            team_picks['dire4'] = matches['pick7']
            team_picks['dire5'] = matches['pick9']
            team_picks['radiant_win'] = matches['radiant_win']
#             a = pd.DataFrame(matches[['radiant_id', 'dire_id','pick2','pick3','pick5','pick8','pick10',
#                                       'pick1','pick4','pick6','pick8','pick9']].iloc[index])
            #print(a)
        #pd.concat([team_picks, a])
        
    return team_picks

In [21]:
dataMatches_team_picks = create_df_team_picks(matches)
dataMatches_team_picks.to_csv('table wtih team, kills, picks path 7.06.csv')

In [24]:
# Разделение данных бех перемешивания
X, y = np.split(dataMatches_team_picks, [100]) 

In [25]:
# Разделенеие на признаки и классы
X_train = X.drop(['radiant_win'], axis=1)
y_train = X['radiant_win']

X_test = y.drop(['radiant_win'], axis=1)
y_test = y['radiant_win']

In [26]:
forest_last = RandomForestClassifier(n_estimators=36, random_state=40, 
                                                        max_depth=14, max_features=8)
forest_last.fit(X_train, y_train)
print (forest_last.score(X_train, y_train))
print (forest_last.score(X_test, y_test))


1.0
0.521292217327
